En el presente documento se estará realizando un análisis de sentimiento a las conferencias diarias que el president de México lleva a cabo diariamente. Se utilizarán específicamente las comprendidas entre el 1 de Agosto de 2019 y el 30 de Abril de 2021.

Primero realizamos la importación de las librerías que se estarán utilizando durante el procesamiento.

In [5]:
import string
import matplotlib
from nltk import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

Por el momento se ha definido la utilización de un archivo de texto como fuente, por lo que procedemos a su apertura.

In [6]:
# Archivo de texto a utilizar como fuente
abrir_archivo = "/home/robert/GitHub/pfmUNIR/archivos/ConferenciasMatutinas.txt"

with open(abrir_archivo, 'r') as file:
    text = file.read()

Continuamos con actividades de preparación del texto como es la separación en palabras, conversión a minúsculas, eliminación de signos de puntuación, etc.

In [7]:
# Separar palabras
tokens = word_tokenize(text)

# Transformar a minúsculas
tokens = [w.lower() for w in tokens]

# Quitar puntuación
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]

# Quitar otros caracteres no alfanuméricos
words = [word for word in stripped if word.isalpha()]

# Quitar palabras comunes (stop words)
stop_words = set(stopwords.words('spanish'))
words = [w for w in words if not w in stop_words]

A continuación podemos comenzar a utilizar el texto transformado para realizar algunos análisis, podemos empezar con obtener la lista de las palabras más usuadas y graficarlas.

In [8]:
# Obtener la frecuencia de las palabras
freq_dist = FreqDist(words)

# Graficar las 25 más utilizadas
freq_dist.plot(25, cumulative=False)

<Figure size 640x480 with 1 Axes>